In [1]:
!git pull
!pip install shap
import os
os.chdir("/content/Inflation_master_thesis/FEDFormer")

fatal: not a git repository (or any of the parent directories): .git


In [2]:
from utils.tools import dotdict
import torch
from exp.exp_main import Exp_Main
import pandas as pd
import shutil
import os
from tqdm.auto import tqdm
torch.cuda.empty_cache()

In [3]:
args = dotdict()

args.model = 'FEDformer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]
args.task_id = 'informer'
args.is_training = 1
args.version = 'Fourier'
args.task = 'test'
args.data = 'custom' # data
args.root_path = './dataset/' # root path of data file
args.data_path = 'Informer-data.csv' # data file
args.features = 'M' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
args.target = 'Austria' # target feature in S or MS task
args.continent = 'Europe'
args.freq = 'm' # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
args.detail_freq = args.freq
args.freq = args.freq[-1:]
args.checkpoints = './Autoformer_checkpoints' # location of model checkpoints

args.mode_select = 'random'
args.modes = 32
args.seq_len = 32 # input sequence length of Informer encoder
args.label_len = 16# start token length of Informer decoder
args.pred_len = 12 # prediction sequence length
# Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

args.enc_in = 97 # encoder input size
args.dec_in = 97 # decoder input size
args.c_out  = 97 # output size
args.factor = 2 # probsparse attn factor
args.d_model = 512 # dimension of model
args.n_heads = 8 # num of heads
args.e_layers = 2 # num of encoder layers
args.d_layers = 2 # num of decoder layers
args.d_ff = 2048 # dimension of fcn in model
args.dropout = 0.1 # dropout
args.attn = 'prob' # attention used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = True # whether to use distilling in encoder
args.output_attention = False # whether to output attention in ecoder
args.mix = True
args.padding = 0
args.do_predict = True
args.moving_avg = 12
args.cross_activation = 'tanh'
args.base = 'legendre'

args.batch_size = 16
args.learning_rate = 0.0001
args.loss = 'mse'
args.lradj = 'type1'
args.use_amp = False # whether to use automatic mixed precision training

args.num_workers = 0
args.itr = 1
args.train_epochs = 8
args.patience = 3
args.des = 'exp'

args.use_gpu = True if torch.cuda.is_available() else False
args.gpu = 0

args.use_multi_gpu = False
args.devices = '0,1,2,3'

args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

In [4]:
Exp = Exp_Main
seq_lengths = [6]#, 12, 24, 48]

args.task_id = "Autoformer_h1"

if args.is_training:
    for m in ['Autoformer']:
        args.model = m
        for i in tqdm(range(0, 238)):
            best_loss = float('inf')
            for kernel_size in [5]:#, 9, 13, 25]:
                args.moving_avg = kernel_size
                for idx, seq_length in enumerate(seq_lengths):
                    args.seq_len = seq_length
                    for label_length in seq_lengths[:idx + 1]:
                        args.label_len = label_length
                        # Set augments by using data name
                        args.idx = i
                        j = str(i)
                        while(len(j)<3):
                            j = '0' + j
                        for ii in range(args.itr):
                            # setting record of experiments
                            setting = '{}_{}_seqlen{}_labellen{}_heads{}_encoderlayers{}_dm{}_ma{}_{}'.format(
                                j,
                                args.task_id,
                                args.seq_len,
                                args.label_len,
                                args.n_heads,
                                args.e_layers,
                                args.d_model,
                                args.moving_avg,
                                ii)

                            exp = Exp(args)  # set experiments
                            print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
                            try:
                                exp.train(setting)
                            except Exception as e:
                                print(f"===========================GOT ERROR: {e}")
                                continue
                            vali_data, vali_loader = exp._get_data(flag='val')
                            loss = exp.vali(vali_data, vali_loader, exp._select_criterion())
                            loss1 = exp.vali(vali_data, vali_loader, exp._select_criterion())
                            print(f"Validation Loss {loss}, loss1 {loss1}")
                            torch.cuda.empty_cache()
                            if loss < best_loss:
                                print(f"New best model with loss: {loss}")
                                best_loss = loss
                                best_args = args
                                best_setting = setting
                            print(best_loss)
                            print(best_args)
                            print(best_setting)

            if args.do_predict:
                #exp = Exp(best_args)  # set experiments
                print('>>>>>>>start prediction_training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(best_setting))
                #exp.train(best_setting)
                print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(best_setting))
                exp.predict(best_setting, True)
                for ckpt in os.listdir(r"/content/Inflation_master_thesis/FEDFormer/Autoformer_checkpoints"):
                    shutil.rmtree(rf'/content/Inflation_master_thesis/FEDFormer/Autoformer_checkpoints/{ckpt}')

  0%|          | 0/238 [00:00<?, ?it/s]

Use GPU: cuda:0
Autocorrelation used !
Autocorrelation used !
Autocorrelation used !
Autocorrelation used !
Autocorrelation used !
Autocorrelation used !
>>>>>>>start training : 000_Autoformer_h1_seqlen6_labellen6_heads8_encoderlayers2_dm512_ma5_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 199
val 13
test 0
Epoch: 1 cost time: 1.0992863178253174
Epoch: 1, Steps: 13 | Train Loss: 1.3225446 Vali Loss: 1.0632658 Test Loss: nan
Validation loss decreased (inf --> 1.063266).  Saving model ...
Updating learning rate to 0.0001
Epoch: 2 cost time: 0.6110348701477051
Epoch: 2, Steps: 13 | Train Loss: 1.1079895 Vali Loss: 0.9915426 Test Loss: nan
Validation loss decreased (1.063266 --> 0.991543).  Saving model ...
Updating learning rate to 5e-05
Epoch: 3 cost time: 0.5690102577209473
Epoch: 3, Steps: 13 | Train Loss: 0.9968145 Vali Loss: 0.9687145 Test Loss: nan
Validation loss decreased (0.991543 --> 0.968714).  Saving model ...
Updating learning rate to 2.5e-05
Epoch: 4 cost time: 0.5800209045410156
Epoch

KeyboardInterrupt: 

In [ ]:
Exp = Exp_Main
seq_lengths = [6, 12, 24, 48]

args.task_id = "Autoformer_h6"
args.pred_len = 6

if args.is_training:
    for m in ['Autoformer']:
        args.model = m
        for i in tqdm(range(0, 238)):
            best_loss = float('inf')
            for kernel_size in [5, 9, 13, 25]:
                args.moving_avg = kernel_size
                for idx, seq_length in enumerate(seq_lengths):
                    args.seq_len = seq_length
                    for label_length in seq_lengths[:idx + 1]:
                        args.label_len = label_length
                        # Set augments by using data name
                        args.idx = i
                        j = str(i)
                        while(len(j)<3):
                            j = '0' + j
                        for ii in range(args.itr):
                            # setting record of experiments
                            setting = '{}_{}_seqlen{}_labellen{}_heads{}_encoderlayers{}_model{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}__ma{}_{}'.format(
                                j,
                                args.task_id,
                                args.seq_len,
                                args.label_len,
                                args.n_heads,
                                args.e_layers,
                                args.model,
                                args.batch_size,
                                args.mode_select,
                                args.modes,
                                args.data,
                                args.features,
                                args.pred_len,
                                args.d_model,
                                args.d_layers,
                                args.d_ff,
                                args.factor,
                                args.embed,
                                args.distil,
                                args.des,
                                args.moving_avg,
                                ii)

                            exp = Exp(args)  # set experiments
                            print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
                            try:
                                exp.train(setting)
                            except Exception as e:
                                print(f"===========================GOT ERROR: {e}")
                                continue
                            vali_data, vali_loader = exp._get_data(flag='val')
                            loss = exp.vali(vali_data, vali_loader, exp._select_criterion())
                            torch.cuda.empty_cache()
                            if loss < best_loss:
                                print(f"New best model with loss: {loss}")
                                best_loss = loss
                                best_args = args
                                best_setting = setting
                            print(best_loss)
                            print(best_args)
                            print(best_setting)

            if args.do_predict:
                #exp = Exp(best_args)  # set experiments
                print('>>>>>>>start prediction_training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(best_setting))
                #exp.train(best_setting)
                print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(best_setting))
                exp.predict(best_setting, True)
                for ckpt in os.listdir("C:/Users/osahl/Desktop/UNI/master thesis/FEDFormer/Autoformer_checkpoints"):
                    shutil.rmtree(f'C:/Users/osahl/Desktop/UNI/master thesis/FEDFormer/Autoformer_checkpoints/{ckpt}')

In [ ]:
Exp = Exp_Main
seq_lengths = [6, 12, 24, 48]

args.task_id = "Autoformer_h12"
args.pred_len = 12

if args.is_training:
    for m in ['Autoformer']:
        args.model = m
        for i in tqdm(range(0, 238)):
            best_loss = float('inf')
            for kernel_size in [5, 9, 13, 25]:
                args.moving_avg = kernel_size
                for idx, seq_length in enumerate(seq_lengths):
                    args.seq_len = seq_length
                    for label_length in seq_lengths[:idx + 1]:
                        args.label_len = label_length
                        # Set augments by using data name
                        args.idx = i
                        j = str(i)
                        while(len(j)<3):
                            j = '0' + j
                        for ii in range(args.itr):
                            # setting record of experiments
                            setting = '{}_{}_seqlen{}_labellen{}_heads{}_encoderlayers{}_model{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}__ma{}_{}'.format(
                                j,
                                args.task_id,
                                args.seq_len,
                                args.label_len,
                                args.n_heads,
                                args.e_layers,
                                args.model,
                                args.batch_size,
                                args.mode_select,
                                args.modes,
                                args.data,
                                args.features,
                                args.pred_len,
                                args.d_model,
                                args.d_layers,
                                args.d_ff,
                                args.factor,
                                args.embed,
                                args.distil,
                                args.des,
                                args.moving_avg,
                                ii)

                            exp = Exp(args)  # set experiments
                            print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
                            try:
                                exp.train(setting)
                            except Exception as e:
                                print(f"===========================GOT ERROR: {e}")
                                continue
                            vali_data, vali_loader = exp._get_data(flag='val')
                            loss = exp.vali(vali_data, vali_loader, exp._select_criterion())
                            torch.cuda.empty_cache()
                            if loss < best_loss:
                                print(f"New best model with loss: {loss}")
                                best_loss = loss
                                best_args = args
                                best_setting = setting
                            print(best_loss)
                            print(best_args)
                            print(best_setting)

            if args.do_predict:
                #exp = Exp(best_args)  # set experiments
                print('>>>>>>>start prediction_training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(best_setting))
                #exp.train(best_setting)
                print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(best_setting))
                exp.predict(best_setting, True)
                for ckpt in os.listdir("C:/Users/osahl/Desktop/UNI/master thesis/FEDFormer/Autoformer_checkpoints"):
                    shutil.rmtree(f'C:/Users/osahl/Desktop/UNI/master thesis/FEDFormer/Autoformer_checkpoints/{ckpt}')